# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Client-Server Architecture

**Subtopic** - Designing a Scalable Client-Server Application

**Use Case** - Develop a simple multi-threaded server in Python that can handle multiple client connections concurrently. The server should be able to accept, process, and respond to client requests. The server should be designed in a way that it can be scaled up to handle an increasing number of client connections. Use the 'socket' and 'threading' modules from the Python standard library for this task.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

Prompt
Problem Description
Design and implement a simple, scalable, multi-threaded TCP server in Python. The server listens on a configurable host and port. When a client connects, it spawns a new thread to handle that connection. Each client thread reads UTF-8 lines terminated by newline, processes each line, and sends back a response line. If a client sends “shutdown” the server should close only that client’s connection. If any client sends “quit\_server” the server should stop accepting new connections, wait for all active threads to finish, then exit cleanly.

Input Format and Constraints
Clients connect via TCP and send UTF-8 text messages, one per line, ending with `\n`.
Each message is either:

* an arbitrary text line to be echoed back prefixed with `ACK: `
* the literal line `shutdown`
* the literal line `quit_server`

You may assume clients always send well-formed UTF-8 lines. There is no need to validate message syntax beyond detecting these keywords.

Expected Output Format
For a normal text line X, reply with:

```
ACK: X
```

terminated by `\n`.
For `shutdown` from a client, close that client’s socket without sending a reply.
For `quit_server` from any client, close that client and all others, stop accepting new clients, then exit the server process.

Examples

* Client sends `hello world\n`-> server replies `ACK: hello world\n`
* Client sends `shutdown\n`-> server closes that connection
* Client A sends `quit_server\n`-> server closes A and all other connections, then exits




# Requirements

# Explicit and Implicit Points
• Use only the Python standard library modules `socket` and `threading`
• Accept and handle each client in its own `Thread`
• Read and write UTF-8 lines terminated by newline
• Detect `shutdown` and `quit_server` commands exactly as specified
• Ensure the main accept loop runs independently of client threads
• Use a thread-safe flag or event to coordinate full server shutdown

# Solution Expectations
• A function `start_server(host: str, port: int) -> None` that starts the server and blocks until shutdown
• Per-connection logic encapsulated in a `ClientHandler` class (subclass of `threading.Thread` or similar)
• When `quit_server` arrives, no new clients are accepted and the server waits for all client threads to finish before exiting
• Socket errors or client disconnects must be caught and logged per client without crashing the server

# Signatures of Expected Functions

```
def start_server(host: str, port: int) -> None
```

```
class ClientHandler(threading.Thread):
    def __init__(self, conn: socket.socket, addr: tuple[str,int],
                 shutdown_event: threading.Event) -> None
    def run(self) -> None
```

# Relevant Classes or Structures
ClientHandler
• public fields: `conn`, `addr`, `shutdown_event`
• public methods: `__init__`, `run`

# Edge Case Behavior
• Empty input lines-> treat as normal text and reply `ACK: \n`
• Client disconnects mid-line or sends invalid UTF-8-> catch exception, log it, close that client only
• Multiple clients send `quit_server` concurrently-> only the first triggers shutdown, subsequent ones are ignored because the server is already stopping

# Constraints
• No external dependencies beyond `socket`, `threading`, `logging`, and other standard modules
• Follow PEP 8 naming and layout
• No global mutable state except a thread-safe shutdown flag or `Event`
• The server must handle at least dozens of concurrent clients without code changes

# Important Notes
• You can assume the input lines satisfy the format; no extra validation is required
• On detecting invalid data or socket errors per client, simply log and close that client socket
• Keep the prompt and requirements clear to enable straightforward automated testing of each behavior

In [ ]:
# code

#!/usr/bin/env python3
"""
server.py

A simple, scalable, multi-threaded TCP server that cleanly shuts down
when any client sends "quit_server".

Usage:
    python server.py 127.0.0.1 8000

Test with netcat:
    nc 127.0.0.1 8000
    hello           → ACK: hello
    shutdown        → closes only your connection

Test with telnet:
    telnet 127.0.0.1 8000
    quit_server     → shuts down entire server
"""

import logging
import socket
import sys
import threading
from typing import List, Tuple

BACKLOG = 100
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)


class ClientHandler(threading.Thread):
    """
    Handle a single client connection in its own thread.

    Echo messages with "ACK: ". Close on "shutdown".
    Trigger full server shutdown on "quit_server".
    """

    def __init__(
        self,
        conn: socket.socket,
        addr: Tuple[str, int],
        shutdown_event: threading.Event
    ) -> None:
        super().__init__()  # non-daemon so we can join on shutdown
        self.conn = conn
        self.addr = addr
        self.shutdown_event = shutdown_event

    def run(self) -> None:
        logging.info("Client connected %s:%d", *self.addr)
        try:
            with self.conn:
                reader = self.conn.makefile("r", encoding="utf-8", newline="\n")
                writer = self.conn.makefile("w", encoding="utf-8", newline="\n")

                for line in reader:
                    if self.shutdown_event.is_set():
                        break

                    msg = line.rstrip("\n")

                    if msg == "shutdown":
                        logging.info("Closing connection to %s:%d", *self.addr)
                        break

                    if msg == "quit_server":
                        logging.info("Shutdown requested by %s:%d", *self.addr)
                        self.shutdown_event.set()
                        break

                    writer.write(f"ACK: {msg}\n")
                    writer.flush()
        except Exception as exc:
            logging.warning("Error with %s:%d: %s", *self.addr, exc)
        finally:
            try:
                self.conn.shutdown(socket.SHUT_RDWR)
            except Exception:
                pass
            self.conn.close()
            logging.info("Client disconnected %s:%d", *self.addr)


def _accept_loop(
    server_sock: socket.socket,
    shutdown_event: threading.Event,
    threads: List[threading.Thread]
) -> None:
    """
    Accept new connections until shutdown_event is set.
    Immediately re-check after each accept to avoid the shutdown race.
    """
    server_sock.settimeout(1.0)

    while not shutdown_event.is_set():
        try:
            conn, addr = server_sock.accept()
        except socket.timeout:
            continue
        except OSError:
            break

        if shutdown_event.is_set():
            conn.close()
            break

        handler = ClientHandler(conn, addr, shutdown_event)
        handler.start()
        threads.append(handler)


def start_server(host: str, port: int) -> None:
    """
    Start the TCP server on the given host and port.
    Blocks until a client sends "quit_server".
    """
    shutdown_event = threading.Event()
    threads: List[threading.Thread] = []

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
        srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        srv.bind((host, port))
        srv.listen(BACKLOG)
        logging.info("Server listening on %s:%d", host, port)

        try:
            _accept_loop(srv, shutdown_event, threads)
        finally:
            # signal all handlers to stop, close listening socket, wait for threads
            shutdown_event.set()
            srv.close()
            for t in threads:
                t.join()
            logging.info("Server shutdown complete")


if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python server.py <host> <port>")
        sys.exit(1)

    try:
        host_arg = sys.argv[1]
        port_arg = int(sys.argv[2])
    except ValueError:
        print("Port must be an integer.")
        sys.exit(1)

    start_server(host_arg, port_arg)


In [ ]:
# tests


# test_server_full.py

import importlib.util
import os
import socket
import threading
import time
import unittest

# dynamically load our implementation from main.py (or server.py)
MODULE_PATH = os.path.join(os.getcwd(), "main.py")  # adjust if your file is named server.py
spec = importlib.util.spec_from_file_location("app", MODULE_PATH)
app = importlib.util.module_from_spec(spec)
spec.loader.exec_module(app)


class FullServerTest(unittest.TestCase):
    def setUp(self):
        # pick an ephemeral port
        listener = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        listener.bind(("127.0.0.1", 0))
        self.host, self.port = listener.getsockname()
        listener.close()

        # start the server in its own thread
        self.server_thread = threading.Thread(
            target=app.start_server,
            args=(self.host, self.port),
            daemon=True
        )
        self.server_thread.start()

        # give it a moment to bind & listen
        time.sleep(0.1)

    def tearDown(self):
        # if it's still running, trigger shutdown
        if self.server_thread.is_alive():
            try:
                s = socket.create_connection((self.host, self.port), timeout=1)
                s.sendall(b"quit_server\n")
                s.close()
            except Exception:
                pass
            # wait at most 1s for it to finish
            self.server_thread.join(timeout=1)

    def _connect(self):
        return socket.create_connection((self.host, self.port), timeout=1)

    def test_echo_and_client_shutdown(self):
        s = self._connect()
        r = s.makefile("r", encoding="utf-8")
        w = s.makefile("w", encoding="utf-8")

        # normal echo
        w.write("hello world\n")
        w.flush()
        self.assertEqual(r.readline(), "ACK: hello world\n")

        # per-client shutdown should close only this socket
        w.write("shutdown\n")
        w.flush()
        time.sleep(0.05)
        self.assertEqual(r.readline(), "")  # EOF
        s.close()

    def test_empty_line(self):
        s = self._connect()
        r = s.makefile("r", encoding="utf-8")
        w = s.makefile("w", encoding="utf-8")

        w.write("\n")
        w.flush()
        self.assertEqual(r.readline(), "ACK: \n")

        w.write("shutdown\n")
        w.flush()
        s.close()

    def test_global_quit_server(self):
        # open two clients
        s1 = self._connect()
        s2 = self._connect()
        r1 = s1.makefile("r", encoding="utf-8")
        w1 = s1.makefile("w", encoding="utf-8")
        r2 = s2.makefile("r", encoding="utf-8")
        w2 = s2.makefile("w", encoding="utf-8")

        # trigger full shutdown
        w1.write("quit_server\n")
        w1.flush()

        # both should see EOF
        time.sleep(0.05)
        self.assertEqual(r1.readline(), "")
        self.assertEqual(r2.readline(), "")

        # wait for server thread to exit
        self.server_thread.join(timeout=1)
        self.assertFalse(self.server_thread.is_alive())

        s1.close()
        s2.close()

    def test_malformed_utf8_closes_client(self):
        s = self._connect()
        # send invalid UTF-8
        s.sendall(b"\xff\xfe\xfd\n")
        # server should detect decode error and close that socket
        time.sleep(0.05)
        data = s.recv(1024)
        self.assertEqual(data, b"")  # closed, no data
        s.close()


if __name__ == "__main__":
    unittest.main()




# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/4448e0ef-c335-4c05-9bb7-325b22217260?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67988%2Fview

#### Model code:

```python
import socket
import threading
import logging

# Configure logging to display thread names and timestamps
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(threadName)s] %(message)s')


class ClientHandler(threading.Thread):
    def __init__(self, conn: socket.socket, addr: tuple[str, int], shutdown_event: threading.Event):
        super().__init__(daemon=True)  # <Issue>: daemon=True—daemon threads may be killed on exit; P/R requires waiting for all threads to finish.
        self.conn = conn
        self.addr = addr
        self.shutdown_event = shutdown_event

    def run(self) -> None:
        with self.conn:
            logging.debug(f'Connected to {self.addr}')
            while not self.shutdown_event.is_set():
                try:
                    data = self.conn.recv(1024).decode('utf-8')
                    if not data:
                        break
                    # <Issue>: splitting on '\n' after arbitrary recv() can split lines incorrectly or drop partial data;
                    # P/R expects reading full lines (e.g., via makefile.readline()).
                    lines = data.split('\n')
                    for line in lines:
                        if line == "":
                            continue

                        if line == 'shutdown':
                            logging.debug(f'Shutting down connection with {self.addr}')
                            return

                        if line == 'quit_server':
                            self.shutdown_event.set()
                            logging.debug('Received quit_server command')
                            return

                        response = f'ACK: {line}\n'
                        self.conn.sendall(response.encode('utf-8'))
                except (ConnectionError, UnicodeDecodeError) as e:
                    logging.error(f'Error with {self.addr}: {e}')
                    break

        logging.debug(f'Disconnected from {self.addr}')


def start_server(host: str, port: int) -> None:
    shutdown_event = threading.Event()

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_sock:
        # <Issue>: missing SO_REUSEADDR—without it binding to the same port after restart can fail.
        server_sock.bind((host, port))
        # <Issue>: no backlog parameter passed; P/R suggests configuring a backlog (e.g., listen(BACKLOG)).
        server_sock.listen()
        logging.info(f'Server started on {host}:{port}')

        threads = []
        while not shutdown_event.is_set():
            try:
                server_sock.settimeout(1.0)  # Timeout the accept call periodically to check for shutdown
                try:
                    conn, addr = server_sock.accept()
                except socket.timeout:
                    continue

                # <Issue>: no re-check of shutdown_event immediately after accept()—race can accept one extra connection after quit_server.
                handler = ClientHandler(conn, addr, shutdown_event)
                handler.start()
                threads.append(handler)
            except Exception as e:
                logging.error(f'Error accepting connections: {e}')
                break

        logging.debug('Server shutdown initiated, waiting for threads to finish')
        for t in threads:
            # <Issue>: t.join() without timeout is correct per P/R, but join on daemon threads is ineffective.
            t.join()


if __name__ == '__main__':
    start_server('127.0.0.1', 12345)
```